# Query Milvus

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [3]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os

import time

import pymilvus

from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [4]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("pymilvus version: " + pymilvus.__version__)

numpy version: 1.23.5
pandas version: 2.0.2
pymilvus version: 2.2.9


### Option 2 - push data from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. You utilize the `mode=stage` flag in the in the `| fit` command to do this. The search results are accessible then as csv file with the same model name that is defined in the `into app:<modelname>` part of the fit statement. Additionally, meta data is retrieved and accessible as json file. In the same way you can further work with the meta data object as it is exposed in the fit and apply function definitions below in stage 3 and 4.

| makeresults count=10<br>
| streamstats c as i <br>
| eval s = i%3 <br>
| eval feature_{s}=0 <br>
| foreach feature_* [eval \<\<FIELD\>\>=random()/pow(2,31)]<br>
| fit MLTKContainer mode=stage algo=barebone_template _time feature_* i into app:barebone_template<br>

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [5]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [6]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("query_milvus")

In [7]:
df

,vector_1,vector_2,vector_3,vector_4
0,0.702,0.117,0.571,0.552
1,0.588,0.696,0.511,0.925
2,0.325,0.067,0.410,0.379


In [27]:
param

{'options': {'params': {'mode': 'stage',
   'algo': 'query_milvus',
   'collection_name': 'new_collection',
   'n_dims': '4'},
  'args': ['vector_*'],
  'feature_variables': ['vector_*'],
  'model_name': 'query_milvus',
  'algo_name': 'MLTKContainer',
  'mlspl_limits': {'handle_new_cat': 'default',
   'max_distinct_cat_values': '100',
   'max_distinct_cat_values_for_classifiers': '100',
   'max_distinct_cat_values_for_scoring': '100',
   'max_fit_time': '600',
   'max_inputs': '100000',
   'max_memory_usage_mb': '4000',
   'max_model_size_mb': '30',
   'max_score_time': '600',
   'use_sampling': 'true'},
  'kfold_cv': None},
 'feature_variables': ['vector_1', 'vector_2', 'vector_3', 'vector_4']}

## Stage 2 - create and initialize a model

In [8]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
        
    pk_type=DataType.VARCHAR        
    embedding_type=DataType.FLOAT_VECTOR
    
    try:
        collection_name=param['options']['params']['collection_name']
    except:
        collection_name="default_collection"
    
    print("start connecting to Milvus")
    # this hostname may need changing to a specific local docker network ip address depending on docker configuration
    connections.connect("default", host="milvus-standalone", port="19530")

    collection_exists = utility.has_collection(collection_name)
    
    if collection_exists:
        print(f"The collection {collection_name} already exists")
        collection = Collection(collection_name)
        collection.load()
    else:
        print(f"The collection {collection_name} does not exist")
        raise Exception("The collection {collection_name} does not exist. Create it by sending data to a collection with that name using the push_to_milvus algo.")
    
    model['collection']=collection
    model['collection_name']=collection_name
    
    return model

In [9]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

start connecting to Milvus
The collection new_collection already exists
{'collection': <Collection>:
-------------
<name>: new_collection
<description>: dsdl schema for new_collection
<schema>: {'auto_id': True, 'description': 'dsdl schema for new_collection', 'fields': [{'name': '_key', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'embeddings', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 4}}, {'name': 'label', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000}}]}
, 'collection_name': 'new_collection'}


## Stage 3 - fit the model

In [10]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    try:
        n_neighbours=int(param['options']['params']['n_neighbours'])
    except:
        n_neighbours=3
        
    try:
        splitter=int(param['options']['params']['splitter'])
    except:
        splitter="|"
    
    vectors_to_search = df.values.tolist()
    
    search_params = {
        "metric_type": "L2",
        "params": {"nprobe": 10},
    }
    
    result = pd.DataFrame(model['collection'].search(vectors_to_search, "embeddings", search_params, limit=n_neighbours, output_fields=["_key","label"]))
    
    def expand_hits(element):
        label=element.entity.get('label')
        #distance=element.distance
        return label
    
    result=result.applymap(expand_hits).astype(str)
    result=pd.DataFrame(result.apply(lambda row: f'{splitter}'.join(row.values), axis=1))
    
    result.columns=['neighbours']
    
    model['result']=result
    
    return result

In [11]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

  neighbours
0      2|8|2
1      6|2|3
2      0|0|0


## Stage 4 - apply the model

In [105]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    return model['result']

In [103]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

  neighbours
0      2,0,3
1      3,7,0
2      0,1,6


## Stage 7 - provide a summary of the model

In [17]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns